In [ ]:
#allows plots in notebook
%matplotlib inline
#sets notebook figures to high quality svg
%config InlineBackend.figure_format = 'svg'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

#importing ODElib for this demo
import ODElib #current build 0.1.1

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Demo notebook for ODElib - Modeling Infection States (updated 2-12-2021)
This notebook is meant to demonstrate the versatility of ODElib by investigating the performance of different ODEs with a varying number of infection states. We will guide users through this executable notebook and highlight everything they need to know to get to analyzing their own data!

# Writting your models for ODElib

ODElib requires minimal configuration to start analyzing models, however, it is extremely important that the models, state variables, and parameters are clearly defined. Under the hood, ODElib uses [scipy's integrate module](https://docs.scipy.org/doc/scipy/reference/integrate.html) to numerically solve the ODE. ODElib automates several aspects of using this API as to focus on model testing rather than writing code to eventually test models. As such, ODElib requries only a few key arguments to automate these process. Foremost, ODElib requires a ODE written as a python function, a list of parameter names that matches the unpacking order in the ODE python funciton, and a list of state variable names that matches the unpacking order in the ODE python function. 

As a demo, we will test the performance of several viral/host infection models. Each model makes a different assumption about the number of infection states in the system. 

In [ ]:
def zero_i(y,t,ps):
    '''
    Viral/Host interaction model with zero infected classes
    
    State Variables: S (susceptible) and V (virus)
    
    Parameters: mu (growth rate), phi (infection rate), beta (burst size)
    '''
    #Unpacking of parameters
    mu,phi,beta=ps[0],ps[1],ps[2]
    #Unpacking of State Variables
    S,V = y[0],y[1]
    #differntial of the susceptible host population
    dSdt = mu*S - phi*S*V
    #differntial of the viral population
    dVdt = beta*phi*S*V - phi*S*V
    return np.array([dSdt,dVdt])

def zero_i_(y,t,ps):
    '''
    Viral/Host interaction model with zero infected classes
    
    State Variables: S (susceptible) and V (virus)
    
    Parameters: mu (growth rate), phi (infection rate), beta (burst size)
    '''
    #Unpacking of parameters
    mu_max,km,phi,beta=ps[0],ps[1],ps[2]
    #Unpacking of State Variables
    S,V = y[0],y[1]
    #differntial of the susceptible host population
    dSdt = mu_max*(S/(km+S)) - phi*S*V
    #differntial of the viral population
    dVdt = beta*phi*S*V - phi*S*V
    return np.array([dSdt,dVdt])



# one infected classes
def one_i(y,t,ps):
    '''
    Viral/Host interaction model with one infected class
    
    State Variables: S (susceptible), V (virus), and I# is the infected state
    
    Parameters: mu (growth rate), phi (infection rate), beta (burst size), lam (lysis rate)
    '''
    mu,phi,beta,lam=ps[0],ps[1],ps[2],ps[3]
    #we are now unpacking thre  
    S,I1,V = y[0],y[1],y[2]
    dSdt = mu*S - phi*S*V
    dI1dt = phi*S*V - lam*I1
    dVdt = beta*lam*I1 - phi*S*V
    return np.array([dSdt,dI1dt,dVdt])



In [ ]:
df = pd.read_csv("testdat.csv")
df=df.replace({'virus':'V','host':'H'})
df=df.rename({'uncertainty':'sigma'},axis=1)
df['log_sigma'] = ODElib.Statistics.stats.predict_logsigma(df['sigma'],df['abundance'])
df

In [ ]:
mu_p=ODElib.parameter(stats_gen=scipy.stats.lognorm,
                      hyperparameters={'s':3,'scale':1e-8})
#rename phi_prior
phi_p=ODElib.parameter(stats_gen=scipy.stats.lognorm,
                       hyperparameters={'s':3,'scale':1e-8})
beta_p=ODElib.parameter(stats_gen=scipy.stats.lognorm,
                        hyperparameters={'s':1,'scale':25})


zeroI=ODElib.ModelFramework(ODE=zero_i,
                          parameter_names=['mu','phi','beta'],
                          state_names = ['H','V'],
                          dataframe=df,
                          mu = mu_p,
                          phi = phi_p,
                          beta = beta_p,
                          t_steps=288
                         )
f,ax = zeroI.plot()

# Poor fits
We see we have very poor fits. To find better values, we can launch MCMC.

In [ ]:
#chain_inits=1,iterations_per_chain=1000,cpu_cores=1,static_parameters=list(),print_report=True,fitsurvey_samples=1000,sd_fitdistance=3.0
posterior = zeroI.MCMC(chain_inits=32,cpu_cores=8,fitsurvey_samples=10000,sd_fitdistance=3.0)

In [ ]:
posterior

In [ ]:
def plot_histogram(series,logspace=True, name=None):
    '''
    This is a plotting function to help us visualize posterior distributions
    '''
    if logspace:
        axessubplot = series.hist(bins=np.logspace(np.log10(series.min()),
                                                   np.log10(series.max()), 50))
        axessubplot.figure.gca().set_xscale("log")
    else:
        axessubplot = series.hist(bins=np.linspace(series.min(),series.max(), 50))
    if series.name or name:
        if name:
            axessubplot.set_title(name)
        else:
            axessubplot.set_title(series.name)
    return(axessubplot)

In [ ]:
muposterior=plot_histogram(posterior['mu'],name='mu posterior')

In [ ]:
f=phi_p.get_figure(logspace=True)

In [ ]:
phiposterior=plot_histogram(posterior['phi'],name='phi posterior')

In [ ]:
betaposterior=plot_histogram(posterior['beta'],name='beta posterior',logspace=False)

In [ ]:
#grabbing a parameter set from the posterior to show fits are much better
zeroI.set_parameters(**posterior.iloc[-1][zeroI.get_pnames()].to_dict())
zeroI.plot()

# 1 infection state

In [ ]:
mu_p=ODElib.parameter(scipy.stats.lognorm,{'s':3,'scale':1e-8})
phi_p=ODElib.parameter(scipy.stats.lognorm,{'s':3,'scale':1e-8})
beta_p=ODElib.parameter(scipy.stats.lognorm,{'s':.5,'scale':20})
lam_p=ODElib.parameter(scipy.stats.lognorm,{'s':2,'scale':.01})

oneI=ODElib.ModelFramework(ODE=one_i,
                          parameter_names=['mu','phi','beta','lam'],
                          state_names = ['S','I1','V'],
                          dataframe=df,
                          mu = mu_p,
                          phi = phi_p,
                          beta = beta_p,
                          lam=lam_p, 
                          t_steps=1000,
                          state_summations={'H':['S','I1']},
                          S=5236900
                           
                         )
oneI.plot()

In [ ]:
#changed sd_fitdistance to cast wider net
posterior_onei = oneI.MCMC(chain_inits=8,cpu_cores=8,fitsurvey_samples=10000,sd_fitdistance=5.0)
posterior_onei

In [ ]:
oneI.set_parameters(**posterior_onei.iloc[-1][oneI.get_pnames()].to_dict())
oneI.plot()

In [ ]:
muposterior=plot_histogram(posterior_onei['mu'],name='mu posterior (oneI)')

In [ ]:
phiposterior=plot_histogram(posterior_onei['phi'],name='phi posterior (oneI)')

In [ ]:
betaposterior=plot_histogram(posterior_onei['beta'],name='beta posterior (oneI)',logspace=False)

In [ ]:
lamposterior=plot_histogram(posterior_onei['lam'],name='lambda posterior (oneI)')